In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
from matplotlib.gridspec import GridSpec

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
os.chdir('/home2/s439906/project/CASP16/monomer/')




# measures are in  GDT_HA    GDC_SC  reLLG_const       QSE     AL0_P     SphGr    CAD_AA      LDDT  MolPrb_Score
measures = ["GDT_HA", "GDC_SC", "reLLG_const", "QSE", "AL0_P", "SphGr", "CAD_AA", "LDDT", "MolPrb_Score"]
weights = [1/6, 1/16, 1/6,1/6,1/16, 1/8,1/8, 1/16,1/16]
assert sum(weights) == 1
assert len(measures) == len(weights)

# measure = "GDT_TS"
# measure = "GDT_HA"

model = "first"
model = "best"

mode = "easy"
mode = "medium"
mode = "difficult"
# mode = "all"

phase = "1"
phase = "0"
phase = "2"
phase = "0,1,2"

top_n = 14

impute_value = -2



fig_dir = "./CASP16_figures/"
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)
score_dir = "./score_all/"

# sum_data_file = f"sum-{model}-{mode}-impute={impute_value}_unweighted.csv"
# data = pd.read_csv(score_dir+sum_data_file, index_col=0)
# data_columns = data.columns
# target_count = {}
# for EU in data_columns:
#     target = EU.split("-")[0]
#     if target not in target_count:
#         target_count[target] = 0
#     target_count[target] += 1
# target_weight = {key: 1/value for key, value in target_count.items()}
# EU_weight = {EU: target_weight[EU.split("-")[0]]
#              for EU in data_columns}
# EU_weight = pd.Series(EU_weight)


os.chdir('/home2/s439906/project/CASP16/monomer/')
score_df =pd.DataFrame()
for i in range(len(measures)):
    measure = measures[i]
    weight = weights[i]
    data_file = f"{measure}-{model}-{mode}-{phase}-impute={impute_value}-EU_weighted_sum.csv"
    data_df = pd.read_csv(score_dir+data_file, index_col=0)
    data_df = data_df*weight
    # data_df = data_df*EU_weight
    data_sum = data_df.sum(axis=1)
    score_df[measure] = data_sum
score_df["sum"] = score_df.sum(axis=1)
score_df = score_df.sort_values(by="sum", ascending=False)
largest_value = score_df["sum"].max()



# get the top n rows
score_df = score_df.head(top_n)
groups = score_df.index.to_list()
GDT_HA = score_df["GDT_HA"].to_list()
GDC_SC = score_df["GDC_SC"].to_list()
reLLG_const = score_df["reLLG_const"].to_list()
QSE = score_df["QSE"].to_list()
AL0_P = score_df["AL0_P"].to_list()
SphGr = score_df["SphGr"].to_list()
CAD_AA = score_df["CAD_AA"].to_list()
LDDT = score_df["LDDT"].to_list()
MolPrb_Score = score_df["MolPrb_Score"].to_list()


all_measures = [GDT_HA, GDC_SC, reLLG_const, QSE, AL0_P, SphGr, CAD_AA, LDDT, MolPrb_Score]
positive_series = [[val if val > 0 else 0 for val in series] for series in all_measures]
negative_series = [[val if val < 0 else 0 for val in series] for series in all_measures]


if top_n > 15:
    fig = plt.figure(figsize=(7, 6), dpi=300)


    bottom_positive = np.zeros(len(groups))
    bottom_negative = np.zeros(len(groups))

    colors = ['tab:blue', 'tab:green', 'tab:orange', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive']

    for i, series in enumerate(positive_series):
        plt.bar(groups, series, bottom=bottom_positive, color=colors[i], alpha=0.9)
        bottom_positive += series

    for i, series in enumerate(negative_series):
        plt.bar(groups, series, bottom=bottom_negative, color=colors[i], alpha=0.5)
        bottom_negative += series

    plt.title(f'models: {model}, difficulty: {mode}, phase: {phase}, impute:{impute_value}, top_n: {top_n}, ', fontsize=12, pad=20)
    plt.legend(measures, loc='upper right',fontsize=10)
    plt.xticks(rotation=90, fontsize=16)
    # y range : -5 to 95
    # if model == "best":
    #     plt.ylim(-5, 105)
    # elif model == "first":
    #     plt.ylim(-10, 75)
    plt.ylim(-10,largest_value+25)
    plt.ylabel('cumulative z-score', fontsize=16)
    plt.yticks(fontsize=16)
    plt.tight_layout()
    plt.show()
    fig.savefig(fig_dir+f"sum_z_score_{model}_{mode}_{phase}_impute={impute_value}_top_n={top_n}.png")

else:
    fig = plt.figure(figsize=(3.5, 5), dpi=300)
    bottom_positive = np.zeros(len(groups))
    bottom_negative = np.zeros(len(groups))

    colors = ['tab:blue', 'tab:green', 'tab:orange', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive']

    for i, series in enumerate(positive_series):
        plt.bar(groups, series, bottom=bottom_positive, color=colors[i], alpha=0.9)
        bottom_positive += series

    for i, series in enumerate(negative_series):
        plt.bar(groups, series, bottom=bottom_negative, color=colors[i], alpha=0.5)
        bottom_negative += series

    plt.title(f'models: {model}, difficulty: {mode}, phase: {phase}, impute:{impute_value}, top_n: {top_n}, ', fontsize=6, pad=20)
    plt.legend(measures, loc='upper right',fontsize=6)
    plt.xticks(rotation=90, fontsize=12)
    # y range : -5 to 95
    # if model == "best":
    #     plt.ylim(-5, 105)
    # elif model == "first":
    #     plt.ylim(-10, 75)
    plt.ylim(-10,largest_value+25)
    plt.ylabel('cumulative z-score', fontsize=12)
    plt.yticks(fontsize=12)
    plt.tight_layout()
    plt.show()
    fig.savefig(fig_dir+f"sum_z_score_{model}_{mode}_{phase}_impute={impute_value}_top_n={top_n}.png")
score_df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 示例数据
categories = ['TS01', 'TS02', 'TS03', 'TS04', 'TS05']
series1 = [10, 15, -5, 20, -10]
series2 = [5, -10, 15, -5, 10]
series3 = [15, 20, -10, 10, 5]

# 分离正负部分
all_series = [series1, series2, series3]
positive_series = [[val if val > 0 else 0 for val in series] for series in all_series]
negative_series = [[val if val < 0 else 0 for val in series] for series in all_series]

# 绘制堆叠条形图
fig = plt.figure()

# 初始化底部位置
bottom_positive = np.zeros(len(categories))
bottom_negative = np.zeros(len(categories))

# 颜色列表
colors = ['tab:blue', 'tab:green', 'tab:orange']

# 绘制正值部分
for i, series in enumerate(positive_series):
    plt.bar(categories, series, bottom=bottom_positive, color=colors[i], alpha=0.8)
    bottom_positive += series

# 绘制负值部分
for i, series in enumerate(negative_series):
    plt.bar(categories, series, bottom=bottom_negative, color=colors[i], alpha=0.4)
    bottom_negative += series

# 添加标题和图例
plt.title('Stacked Bar Chart with Positive and Negative Values')
plt.legend(['Series 1', 'Series 2', 'Series 3'])

# 显示图表
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 示例数据
categories = ['TS01', 'TS02', 'TS03', 'TS04', 'TS05']
series1 = [10, 15, -5, 20, -10]
series2 = [5, -10, 15, -5, 10]
series3 = [15, 20, -10, 10, 5]

# 分离正负部分
all_series = [series1, series2, series3]
positive_series = [[val if val > 0 else 0 for val in series] for series in all_series]
negative_series = [[val if val < 0 else 0 for val in series] for series in all_series]

# 绘制堆叠条形图
fig, ax = plt.subplots()

# 初始化底部位置
bottom_positive = np.zeros(len(categories))
bottom_negative = np.zeros(len(categories))

# 颜色列表
colors = ['tab:blue', 'tab:green', 'tab:orange']

# 绘制正值部分
for i, series in enumerate(positive_series):
    ax.bar(categories, series, bottom=bottom_positive, color=colors[i], alpha=0.8)
    bottom_positive += series

# 绘制负值部分
for i, series in enumerate(negative_series):
    ax.bar(categories, series, bottom=bottom_negative, color=colors[i], alpha=0.4)
    bottom_negative += series

# 添加标题和图例
ax.set_title('Stacked Bar Chart with Positive and Negative Values')
ax.legend(['Series 1', 'Series 2', 'Series 3'])

# 显示图表
plt.show()